# Vytváranie AI agentov s trvalou pamäťou pomocou Cognee

Tento notebook ukazuje, ako vytvoriť inteligentných AI agentov so sofistikovanými pamäťovými schopnosťami pomocou [**cognee**](https://www.cognee.ai/) - open source AI pamäte, ktorá kombinuje znalostné grafy, sémantické vyhľadávanie a správu relácií na vytváranie kontextovo uvedomelých AI systémov.

## 🎯 Ciele učenia

Na konci tohto tutoriálu budete rozumieť, ako:
- **Vytvárať znalostné grafy podporované embeddingami**: Transformovať nestruktúrovaný text na štruktúrované, dotazovateľné znalosti
- **Implementovať pamäť relácií**: Vytvárať viacotáčkové konverzácie s automatickým uchovávaním kontextu
- **Ukladať konverzácie**: Voliteľne ukladať dôležité interakcie do dlhodobej pamäte na budúce použitie
- **Dotazovať sa pomocou prirodzeného jazyka**: Pristupovať k historickému kontextu a využívať ho v nových konverzáciách
- **Vizualizovať pamäť**: Preskúmať vzťahy v znalostnom grafe vášho agenta


## 🏗️ Čo vytvoríte

V tomto návode si vytvoríme **Asistenta pre programovanie** s trvalou pamäťou, ktorý:

### 1. **Vytváranie znalostnej bázy**
   - Získava informácie o profile a odbornosti vývojára
   - Spracováva princípy a osvedčené postupy programovania v Pythone
   - Ukladá historické konverzácie medzi vývojármi a AI asistentmi

### 2. **Konverzácie s vedomím kontextu**
   - Udržiava kontext naprieč viacerými otázkami v rámci jednej relácie
   - Automaticky ukladá každú dvojicu otázka/odpoveď pre efektívne vyhľadávanie
   - Poskytuje súvislé a kontextové odpovede na základe histórie konverzácie

### 3. **Dlhodobá pamäť**
   - Ukladá dôležité konverzácie do dlhodobej pamäte
   - Získava relevantné spomienky zo znalostnej bázy a minulých relácií na obohatenie nových interakcií
   - Buduje rastúcu znalostnú bázu, ktorá sa časom zlepšuje

### 4. **Inteligentné vyhľadávanie v pamäti**
   - Používa grafovo orientované sémantické vyhľadávanie na nájdenie relevantných informácií v celej uloženéj znalosti
   - Filtruje vyhľadávania podľa podskupín údajov (informácie o vývojárovi vs. princípy)
   - Kombinuje viacero zdrojov údajov na poskytovanie komplexných odpovedí


## 📋 Predpoklady a nastavenie

### Požiadavky na systém

Pred začiatkom sa uistite, že máte:

1. **Prostredie Python**
   - Python 3.9 alebo novší
   - Virtuálne prostredie (odporúčané)
   
2. **Redis Cache** (Povinné pre správu relácií)
   - Lokálny Redis: `docker run -d -p 6379:6379 redis`
   - Alebo použite spravovanú službu Redis
   
3. **Prístup k LLM API**
   - OpenAI API kľúč alebo iní poskytovatelia (pozrite [dokumentáciu](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Konfigurácia databázy**
   - Predvolene nie je potrebná žiadna konfigurácia. Cognee používa databázy založené na súboroch (LanceDB a Kuzu)
   - Voliteľne môžete nastaviť Azure AI Search ako vektorový úložný priestor (pozrite [dokumentáciu](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Konfigurácia prostredia

Vytvorte súbor `.env` vo vašom projektovom adresári s nasledujúcimi premennými:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Pochopenie pamäťovej architektúry Cognee

### Ako funguje Cognee

Cognee poskytuje sofistikovaný pamäťový systém, ktorý presahuje jednoduché ukladanie kľúč-hodnota:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Kľúčové komponenty:

1. **Knowledge Graph**: Ukladá entity, vzťahy a sémantické prepojenia
2. **Vector Embeddings**: Umožňuje sémantické vyhľadávanie vo všetkých uložených informáciách
3. **Session Cache**: Udržiava kontext konverzácie v rámci a medzi reláciami
4. **NodeSets**: Organizujú dáta do logických kategórií pre cielené vyhľadávanie

### Typy pamäte v tomto tutoriáli:

- **Persistent Memory**: Dlhodobé ukladanie v knowledge graph
- **Session Memory**: Dočasný kontext konverzácie v Redis cache
- **Semantic Memory**: Vyhľadávanie na základe podobnosti vektorov naprieč všetkými dátami


## 📦 Inštalácia potrebných balíkov

Nainštalujte Cognee s podporou Redis pre správu relácií:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inicializácia prostredia a načítanie knižníc

Uistite sa, že:
1. Redis je spustený (napr. cez Docker: `docker run -d -p 6379:6379 redis`)
2. Premenné prostredia sú nastavené pred importovaním modulov cache
3. Ak je to potrebné, reštartujte jadro a spustite bunky v poradí

Nasledujúca bunka:
1. Načíta premenné prostredia zo súboru `.env`
2. Nakonfiguruje Cognee s vašimi nastaveniami LLM
3. Povolenie cache pre správu relácií
4. Overí, či sú všetky komponenty správne pripojené


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfigurácia úložných adresárov

Cognee používa dva samostatné adresáre pre svoje operácie:
- **Data Root**: Ukladá importované dokumenty a spracované dáta
- **System Root**: Obsahuje databázu znalostného grafu a systémové metadáta

Pre tento návod vytvoríme izolované adresáre nasledovne:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Obnoviť stav pamäte

Predtým, než začneme budovať náš pamäťový systém, uistime sa, že začíname od začiatku.

> 💡 **Tip**: Tento krok môžete preskočiť, ak chcete zachovať existujúce spomienky z vašich predchádzajúcich spustení, keď budete tento notebook používať neskôr.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Časť 1: Budovanie znalostnej bázy

### Zdroje dát pre nášho asistenta vývojára

Na vytvorenie komplexnej znalostnej bázy použijeme tri typy dát:

1. **Profil vývojára**: Osobná odbornosť a technické zázemie
2. **Najlepšie praktiky v Pythone**: Zen of Python s praktickými usmerneniami
3. **Historické konverzácie**: Minulé Q&A relácie medzi vývojármi a AI asistentmi

Táto rôznorodá dátová základňa umožní nášmu agentovi:
- Pochopiť technický kontext používateľa
- Uplatňovať najlepšie praktiky v odporúčaniach
- Učiť sa z predchádzajúcich úspešných interakcií


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Spracovanie údajov do znalostného grafu

Teraz premeníme náš surový text na štruktúrovanú pamäť. Tento proces:

1. **Pridáva údaje do NodeSets**: Organizuje informácie do logických kategórií
   - `developer_data`: Profil vývojára a konverzácie
   - `principles_data`: Najlepšie postupy a usmernenia pre Python

2. **Spúšťa Cognify Pipeline**: Extrahuje entity, vzťahy a vytvára embeddings
   - Identifikuje kľúčové koncepty
   - Vytvára semantické spojenia medzi súvisiacimi informáciami
   - Generuje vektorové embeddings

Tento proces môže chvíľu trvať, kým LLM spracuje text a vytvorí štruktúru grafu:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Vizualizácia znalostného grafu

Preskúmajme štruktúru nášho znalostného grafu. Vizualizácia zobrazuje:
- **Uzly**: Entity extrahované z textu (koncepty, technológie, osoby)
- **Hrany**: Vzťahy a spojenia medzi entitami
- **Klastre**: Súvisiace koncepty zoskupené podľa sémantickej podobnosti

Otvorte vygenerovaný HTML súbor vo svojom prehliadači, aby ste mohli interaktívne preskúmať graf:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Obohaťte pamäť s Memify

Funkcia `memify()` analyzuje graf znalostí a generuje inteligentné pravidlá o údajoch. Tento proces:
- Identifikuje vzory a osvedčené postupy
- Vytvára praktické usmernenia na základe obsahu
- Ustanovuje vzťahy medzi rôznymi oblasťami znalostí

Tieto pravidlá pomáhajú agentovi robiť informovanejšie rozhodnutia pri odpovedaní na otázky. Zachytenie druhej vizualizácie vám umožní porovnať, ako sa graf zahusťuje po obohatení.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Časť 2: Inteligentné vyhľadávanie v pamäti

### Ukážka 1: Integrácia znalostí z viacerých dokumentov

Teraz, keď je náš graf znalostí vytvorený, otestujeme, ako Cognee kombinuje informácie z viacerých zdrojov na zodpovedanie zložitých otázok.

Prvá otázka demonštruje:
- **Sémantické porozumenie**: Vyhľadávanie relevantných konceptov, aj keď nie sú výslovne spomenuté
- **Krížové odkazy**: Kombinovanie profilu vývojára s princípmi Pythonu
- **Kontextové uvažovanie**: Uplatňovanie najlepších postupov na konkrétne implementácie

### Ukážka 2: Filtrované vyhľadávanie pomocou NodeSets

Druhá otázka ukazuje, ako cieliť na konkrétne podmnožiny grafu znalostí:
- Používa parameter `node_name` na vyhľadávanie iba v rámci `principles_data`
- Poskytuje zamerané odpovede z konkrétnej oblasti znalostí
- Užitočné, keď potrebujete informácie špecifické pre danú oblasť


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Časť 3: Nastavenie správy relácií

### Aktivácia pamäte konverzácie

Správa relácií je kľúčová pre udržanie kontextu počas viacerých interakcií. Tu urobíme:

1. **Inicializácia používateľského kontextu**: Vytvorenie alebo načítanie používateľského profilu na sledovanie relácie
2. **Konfigurácia cache enginu**: Pripojenie k Redis na ukladanie histórie konverzácií
3. **Aktivácia premenných relácie**: Nastavenie kontextových premenných, ktoré pretrvávajú medzi dotazmi

> ⚠️ **Dôležité**: Vyžaduje, aby Redis bežal a `CACHING=true` bolo nastavené vo vašom prostredí


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Pomocná funkcia: Zobrazenie histórie relácie

Táto pomocná funkcia nám umožňuje prezerať históriu konverzácií uloženú v Redis. Je užitočná na:
- Ladenie správy relácií
- Overenie, že konverzácie sú ukladané do vyrovnávacej pamäte
- Pochopenie, aký kontext je dostupný pre agenta


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Relácia 1: Laboratórium podpory asynchrónneho spracovania — Prvá otázka

Začnite reláciu `async-support-lab` otázkou na vzory asyncio vhodné pre telemetriu pre masívny web scraper. Graf už pozná asyncio, aiohttp a monitorovacie postupy, takže odpoveď by mala odrážať predchádzajúce rozhovory a zároveň prispôsobiť odpoveď novej otázke.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Skontrolujte pamäť relácie 1 po prvej výmene

Spustením `show_history(session_1)` ihneď po úvodnej otázke sa potvrdí, že Cognee zapísal do Redisu aj výzvu, aj odpoveď. Mali by ste vidieť jeden záznam s pokynmi pre súbežnosť.


In [ ]:
await show_history(session_1)

## Relácia 1: Pokračovanie o dátových modeloch

Ďalej sa pýtame: „Kedy by som mal zvoliť dataclasses namiesto Pydantic?“ s použitím rovnakého ID relácie. Cognee by mala spojiť princípy Pythonu spolu s predchádzajúcimi rozhovormi o FastAPI, aby poskytla jemné rady – ukazujúc, že kontext sa prenáša v rámci pomenovanej relácie.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Potvrďte, že história relácie 1 obsahuje oba kroky

Ďalšie volanie `show_history(session_1)` by malo odhaliť dva záznamy otázok a odpovedí. To zodpovedá kroku "prehrávania pamäte" v laboratóriu Mem0 a dokazuje, že ďalšie kroky rozširujú ten istý prepis.


In [ ]:
await show_history(session_1)

## Relácia 2: Vlákno kontroly návrhu — Nová relácia

Aby sme ukázali izoláciu medzi vláknami, spustíme `design-review-session` a požiadame o usmernenie pri zaznamenávaní incidentov. Aj keď je základná databáza znalostí rovnaká, nové ID relácie udržiava prepisy oddelené.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Prehľad relácie 2 História

`show_history(session_2)` by mal zobraziť iba dvojicu výzva/odpoveď z dizajnovej recenzie. Porovnajte to s reláciou 1, aby ste zdôraznili, ako Cognee uchováva nezávislé prepisy a zároveň využíva zdieľaný graf znalostí.


In [ ]:
await show_history(session_2)

## Zhrnutie

Gratulujeme! Práve ste svojmu kódovaciemu asistentovi pridali skutočnú dlhodobú pamäťovú vrstvu poháňanú Cognee.

V tomto návode ste vzali surový obsah pre vývojárov (kód, dokumentáciu, rozhovory) a premenili ho na graf + vektorovú pamäť, ktorú váš agent dokáže vyhľadávať, analyzovať a neustále zlepšovať.

Čo ste sa naučili

1. **Od surového textu k AI pamäti**: Ako Cognee spracováva neštruktúrované dáta a premieňa ich na inteligentnú, vyhľadávateľnú pamäť pomocou kombinovanej architektúry vektorov a znalostného grafu.

2. **Obohatenie grafu pomocou memify**: Ako ísť nad rámec základného vytvárania grafov a použiť memify na pridanie odvodených faktov a bohatších vzťahov do vášho existujúceho grafu.

3. **Rôzne stratégie vyhľadávania**: Ako dotazovať pamäť pomocou rôznych typov vyhľadávania (otázky a odpovede s vedomím grafu, dopĺňanie v štýle RAG, prehľady, surové časti, vyhľadávanie kódu atď.) v závislosti od potrieb vášho agenta.

4. **Vizualizácia a prieskum**: Ako kontrolovať a ladiť, čo Cognee vytvoril, pomocou vizualizácií grafov a Cognee UI, aby ste mohli skutočne vidieť, ako je štruktúrované poznanie.

5. **Pamäť s vedomím relácie**: Ako kombinovať kontext konkrétnej relácie s trvalou sémantickou pamäťou, aby si agenti mohli pamätať medzi spusteniami bez úniku informácií medzi používateľmi.


## Kľúčové body
1. Pamäť ako znalostný graf podporovaný embeddingami

    - **Štruktúrované porozumenie**: Cognee kombinuje úložisko vektorov a grafov, takže vaše dáta sú nielen vyhľadateľné podľa významu, ale aj prepojené vzťahmi. Cognee používa databázy založené na súboroch ako predvolené (LanceDB pre vektorové, Kuzu pre grafové databázy).

    - **Vyhľadávanie s ohľadom na vzťahy**: Odpovede môžu byť založené nielen na „podobnom texte“, ale aj na tom, ako sú entity prepojené.

    - **Živá pamäť**: Vrstva pamäte sa vyvíja, rastie a zostáva dotazovateľná ako jeden prepojený graf.

2. Režimy vyhľadávania a uvažovania
    - **Hybridné vyhľadávanie**: Vyhľadávanie kombinuje podobnosť vektorov, štruktúru grafu a uvažovanie LLM, od vyhľadávania surových častí až po odpovede na otázky s ohľadom na graf.

    - **Prispôsobenie režimu úlohe**: Používajte režimy v štýle dokončovania, keď chcete odpovede v prirodzenom jazyku, a režimy častí/zhrnutia/grafu, keď váš agent potrebuje surový kontext alebo chce riadiť vlastné uvažovanie.

3. Personalizovaní agenti s vedomím relácie
    - **Kontext relácie + dlhodobá pamäť**: Cognee uchováva krátkodobý kontext „vlákna“ oddelene od dlhodobej pamäte na úrovni používateľa alebo organizácie.

## Reálne aplikácie

1. **Vertikálni AI agenti**

    Použite vzor z tohto notebooku na vytvorenie doménovo inteligentných kopilotov, ktorí využívajú Cognee ako jadro pre vyhľadávanie a uvažovanie:

- **Kopiloti pre vývojárov**: Asistenti na kontrolu kódu, analýzu incidentov a architektúru, ktorí prechádzajú kód, API, návrhové dokumenty a tikety ako jeden pamäťový graf.

- **Kopiloti pre zákazníkov**: Podporní alebo úspešní agenti, ktorí čerpajú z produktovej dokumentácie, FAQ, poznámok CRM a minulých tiketov s vyhľadávaním s ohľadom na graf a citovanými odpoveďami.

- **Interní expertní kopiloti**: Asistenti pre politiku, právo alebo bezpečnosť, ktorí uvažujú nad prepojenými pravidlami, smernicami a historickými rozhodnutiami namiesto izolovaných PDF.

    Cognee je výslovne navrhnutý ako trvalá, presná pamäť pre AI agentov, poskytujúca živý znalostný graf, ktorý sa integruje za vaším agentom a nahrádza ad-hoc kombinácie úložísk vektorov a vlastného grafového kódu.

2. **Zjednotenie dátových sil do jednej pamäte**

    Rovnaký prístup vám tiež pomáha vytvoriť jednotnú pamäťovú vrstvu naprieč rozptýlenými zdrojmi:

- **Zo sil do jedného grafu**: Načítajte štruktúrované (napr. databázy) a neštruktúrované dáta (napr. dokumenty, chaty) do jedného grafu podporovaného embeddingami, namiesto samostatných indexov pre každý systém.

- **Medzi-zdrojové uvažovanie s citáciami**: Spustite viacstupňové uvažovanie nad všetkým—„spojte“ logy, metriky a dokumenty cez graf—a stále vracajte odpovede s overením pôvodu.

- **Centrá znalostí**: Pre oblasti ako bankovníctvo alebo vzdelávanie sa Cognee už používa na zjednotenie PDF, interných systémov a aplikačných dát do jedného znalostného grafu s vektormi, aby agenti mohli odpovedať na otázky s presným, citovaným kontextom.

## Ďalšie kroky

Implementovali ste základný pamäťový cyklus. Tu sú prirodzené rozšírenia, ktoré si môžete vyskúšať sami (pozrite si [dokumentáciu Cognee](https://docs.cognee.ai/) pre podrobnosti):

1. **Experimentujte s časovým povedomím**: Zapnite časové „cognify“, aby ste extrahovali udalosti a časové pečiatky z textu.

2. **Zavedenie uvažovania riadeného ontológiou**: Definujte OWL ontológiu pre vašu oblasť. Použite podporu ontológie Cognee, aby extrahované entity a vzťahy boli ukotvené v tejto schéme, čím sa zlepší kvalita grafu a odpovede špecifické pre danú oblasť.

3. **Pridajte spätnú väzbu**: Nechajte Cognee upravovať váhy hrán grafu na základe skutočnej spätnej väzby používateľov, aby sa vyhľadávanie časom zlepšovalo namiesto toho, aby zostalo statické.

4. **Ladenie pre personalizáciu a správanie relácie**: Používajte ID používateľov, nájomcov a datasety, aby každý človek alebo tím mal svoj vlastný pohľad na zdieľaný pamäťový engine.

5. **Rozšírenie na zložitejších agentov**: Pripojte Cognee k rámcom agentov na vytvorenie multi-agentových systémov, ktoré všetky zdieľajú rovnakú pamäťovú vrstvu. *Microsoft Agent Framework x Cognee plugin čoskoro dostupný.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
